In [23]:
import pandas as pd
import os, json, joblib
from palmerpenguins import load_penguins
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC


# Primer Taller MLOPS 
Presentador por Jacobo & Javier

In [2]:
penguins_df_raw = load_penguins()
penguins_df_raw.sample(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
114,Adelie,Biscoe,39.6,20.7,191.0,3900.0,female,2009
66,Adelie,Biscoe,35.5,16.2,195.0,3350.0,female,2008
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
187,Gentoo,Biscoe,48.4,16.3,220.0,5400.0,male,2008
226,Gentoo,Biscoe,46.4,15.0,216.0,4700.0,female,2008
20,Adelie,Biscoe,37.8,18.3,174.0,3400.0,female,2007
260,Gentoo,Biscoe,43.3,14.0,208.0,4575.0,female,2009
74,Adelie,Torgersen,35.5,17.5,190.0,3700.0,female,2008
176,Gentoo,Biscoe,42.9,13.1,215.0,5000.0,female,2007
166,Gentoo,Biscoe,45.8,14.6,210.0,4200.0,female,2007


In [3]:
penguins_df_raw.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
year                  0
dtype: int64

## Dataset Primeras Impresiones
Dando un primer vistazo al dataset podemos identificar que tienes la siguientes features:
1. Numericas: bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g, year
2. Categóricas: island, sex
3. Target: species

Inicialmente proponemos el siguiente tratamiento:
1. Imputar numericas con median
2. Imputar categorias con most_frequent
3. OneHot a categoricas

Para el entrenamiento de diversos modelos, realizamos la siguiente propuesta inicialmente, que podria cambiar dependiendo de su performance, aunque como obtener el mejor performance no es el objetivo de este taller probablemente nos mantengamos con estos:
1. Logistic Regression
2. Random Forest
3. SVC
4. Gradient Boosting



Separamos las variables X y el target y

In [ ]:
X = penguins_df_raw.drop("species", axis = 1) #datos de entrada
y = penguins_df_raw["species"] # variable a predecir

In [6]:
X.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [7]:
y.head()

0    Adelie
1    Adelie
2    Adelie
3    Adelie
4    Adelie
Name: species, dtype: str

In [13]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y, #usamos stratify porque species es multiclase (adelie, gentoo, chinstrap), y esto amnetiene la proporcion de clases en train y validation
    random_state=42
)

In [14]:
X_train.shape

(275, 7)

In [15]:
X_val.shape

(69, 7)

In [ ]:
# Pipeline para variables numéricas
numeric_transformer = Pipeline( #un pipeline es simplemente una lista ordenada de pasos que se ejecutan uno tras otro
    steps=[
        ("imputer", SimpleImputer(strategy="median")) #aqui decimos para las variables numericas aplica un imputador que reemplace los valores faltantes con la mediana
    ]
)

#cuando entrenemos el modelo el pipeline:
    #1. calcula la mediana usando solo los datos de entrenamiento
    #2. guarda esa mediana internamente
    #3. cuando llegue un nuevo dato en produccion, usa esa misma mediana

# Pipeline para variables categóricas
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

# Aqui tenemos dos pasos
    #1. si sex es nan, lo reemplaza por el valor mas frecuente
    #2. convierte variables categoricas en columnas binarias | handle_unknown="ignore", si aparece una isla nueva que nuncca vimos la ignora


In [20]:
numeric_features = [
    "bill_length_mm",
    "bill_depth_mm",
    "flipper_length_mm",
    "body_mass_g",
    "year"
]

categorical_features = [
    "island",
    "sex"
]


In [ ]:
preprocessor = ColumnTransformer( #este objeto aplica transformaciones diferentes a diferentes columnas automaticamente
    transformers=[
        ("num", numeric_transformer, numeric_features), # a las columnas numericas aplica numeric_transformer
        ("cat", categorical_transformer, categorical_features) # a las columnas categoricas aplica categorical_transformer
    ]
) 
#junta todo en una sola matriz para el modelo


In [ ]:
models = { #diccionario donde la clave es el nombre del modelo y el valor es la instancia del clasificador, esto para poderlos entrenar en un loop y no tener que reescribir codigo
    "logreg": LogisticRegression(max_iter=1000),
    "rf": RandomForestClassifier(random_state=42),
    "svm": SVC(probability=True, random_state=42),
    "gb": GradientBoostingClassifier(random_state=42),
}

trained_models = {} # diccionario vacio para guardar los modelos ya entrenados

for name, clf in models.items(): #iterar sobre cada modelo, ej: primera iteracion -> name = "logreg", clf = LogisticRegression(max_iter = 1000)
    pipe = Pipeline(steps=[ #creamos un pipeline que hace 1. preprocesamiento y 2. clasificacion
        ("preprocessor", preprocessor),
        ("model", clf)
    ])
    # cuando llegue aqui, internamente el preprocessor aprende: 1. mediandas, moda, categorias para one-hot, 2. transforma los datos, 3. el modelo se entrena con datos transformados
    pipe.fit(X_train, y_train) # en resumen aqui ejecuta, 1. imputacion, 2. enconding, 3. transformacion, 4. entrenamiento del modelo 
    #IMPORTANTE: el preprocesamiento se ajusto SOLO con X_trian para evitar data leakage

    preds = pipe.predict(X_val) #el pipeline transforma automaticamente X_val, el modelo predice
    acc = accuracy_score(y_val, preds) #calculamos accuracy
    #Nota para javier: igual para este taller el accuracy no importa mucho, solo pongamoslo como buena practica

    trained_models[name] = pipe #guardar el pipeline completo entrenado, no solo el clasificador sino preprocessor + modelo

    print(f"{name} -> val accuracy: {acc:.4f}")#mostrar resultados
    #Nota para javier: tenemos overfitting obviamente, pero bueno no vamos a perder tiempo entrenando modelos perfectos porque lo que importa es el deployment

/opt/anaconda3/envs/OAP/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


logreg -> val accuracy: 1.0000
rf -> val accuracy: 1.0000
svm -> val accuracy: 0.7681
gb -> val accuracy: 1.0000


In [ ]:
os.makedirs("models", exist_ok=True)

for name, pipe in trained_models.items():
    joblib.dump(pipe, f"models/{name}.joblib") #guardamos con joblib y no con pickle por que estamos usando el ecosiste de scikit learn y en la documentacion es el standard
                                               # esta mas optimizado para objetos grandes con arrays numpy y eso lo hace mas eficiente con modelos sklearn
registry = {
    "default_model": "rf",
    "available_models": list(trained_models.keys())
}

with open("models/registry.json", "w") as f:
    json.dump(registry, f, indent=2)

print("Modelos guardados en /models")

Modelos guardados en /models
